# DIXIE Fire Burn Severity & Vegeation Recovery

**Purpose:** This script evaluates post-fire burn damages and vegetation regrowth following the 2021 Dixie Fire in Northern California. It uses spectral indices like NBR and NDVI, together with land cover classification data like Dynamic World to estimate burn severity and monitor changes in land cover. Section A and B focuses on burn severity and vegetation recovery respectively.

**Inputs:** 
1. Harmonized Landsat Sentinel (HLS)
2. Dynamic World
3. MTBS Burned Area Boundaries
4. TIGER sounty shapefiles

**Outputs:** 
1. Burn severity classifcation map
2. Burned area by severity class chart
3. Seasonal NDVI time series plot
4. Dynamic World land cover change GIF animation
5. Land cover transitions Sankey plot


In [ ]:
# Import libraries
import ee
import geemap
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import matplotlib.patches as mpatches
import seaborn as sns

# autenticate and initialize GEE
ee.Authenticate()
ee.Initialize()

In [ ]:
# -----------------------------------------------
# Section A: Burn Severity
# -----------------------------------------------

# Define Study Area
counties = ee.FeatureCollection("TIGER/2018/Counties")
cali_counties = counties.filter(ee.Filter.eq("STATEFP", "06"))
affected_counties = cali_counties.filter(
    ee.Filter.inList("NAME", ["Butte", "Plumas", "Lassen", "Shasta", "Tehama"])
)

m = geemap.Map()
m.centerObject(affected_counties)


# Load MTBS Fire Boundaries for 2021
def extract_year(feature):
    year = ee.Date(feature.get("Ig_Date")).get("year")
    return feature.set("FireYear", year)

mtbs_2021_fires = (
    ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")
    .map(extract_year)
    .filter(ee.Filter.eq("FireYear", 2021))
    .filterBounds(affected_counties)
)
fire_bbox = mtbs_2021_fires.geometry().bounds()


# Cloud/snow/shadow masking 
def mask_clouds_snow(image):
    fmask = image.select("Fmask")
    cloud_mask = fmask.bitwiseAnd(1 << 1).eq(0)
    shadow_mask = fmask.bitwiseAnd(1 << 3).eq(0)
    snow_mask = fmask.bitwiseAnd(1 << 4).eq(0)
    return image.updateMask(cloud_mask.And(shadow_mask).And(snow_mask))

# Load and process HLS Data
def load_hls_data(collection_id, start_date, end_date):
    return (
        ee.ImageCollection(collection_id)
        .filterBounds(affected_counties)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt("CLOUD_COVERAGE", 30))
        .map(mask_clouds_snow)
    )

# Select and rename Sentinel-2 band names to match Landsat band names 
def process_s30(image_collection):
    return image_collection.map(lambda img: img.select(
        ["B2", "B3", "B4", "B8", "B12"],
        ["B2", "B3", "B4", "B5", "B7"]
    ))

# Compute NBR
def compute_indices(image):
    nbr = image.normalizedDifference(["B5", "B7"]).rename("NBR")
    return image.addBands(nbr)

# Generate pre- and post-fire composites
pre_l30 = load_hls_data("NASA/HLS/HLSL30/v002", "2020-08-01", "2020-08-31").select(["B2", "B3", "B4", "B5", "B7"])
pre_s30 = process_s30(load_hls_data("NASA/HLS/HLSS30/v002", "2020-08-01", "2020-08-31"))
prefire = compute_indices(pre_s30.median().clip(mtbs_2021_fires))  # l30 wasn't used due to weird-looking post-fire image composite

post_l30 = load_hls_data("NASA/HLS/HLSL30/v002", "2021-10-15", "2021-11-15").select(["B2", "B3", "B4", "B5", "B7"])
post_s30 = process_s30(load_hls_data("NASA/HLS/HLSS30/v002", "2021-10-15", "2021-11-15"))
postfire = compute_indices(post_s30.median().clip(mtbs_2021_fires))  # l30 wasn't used due to weird-looking post-fire image composite

# Calculate dNBR and classify burn severity
pre_nbr = prefire.select("NBR")
post_nbr = postfire.select("NBR")
dNBR = pre_nbr.subtract(post_nbr).multiply(1000).rename("dNBR")
RdNBR = dNBR.divide(pre_nbr.abs().add(0.001).sqrt()).rename("RdNBR")

thresholds = [660, 440, 270, 100, -100, -250, -500]
classified_dNBR = dNBR.lt(thresholds).reduce("sum").toInt()
classified_dNBR_clipped = classified_dNBR.clip(mtbs_2021_fires)

burn_colors = ["#a41fd6", "#ff641b", "#ffaf38", "#fff70b", "#0ae042", "#acbe4d", "#7a8737"]

# RGB composites
prefire_rgb = pre_s30.median().clip(affected_counties).select(["B4", "B3", "B2"])
postfire_rgb = post_s30.median().clip(affected_counties).select(["B4", "B3", "B2"])

# Burn area calculations by severity class
pixel_area_acres = ee.Image.pixelArea().divide(4046.86) # hectares
severity_masks = {
    "Low": dNBR.gte(100).And(dNBR.lt(270)),
    "Moderate-Low": dNBR.gte(270).And(dNBR.lt(440)),
    "Moderate-High": dNBR.gte(440).And(dNBR.lt(660)),
    "High": dNBR.gte(660)
}

burn_area_stats = {}
for name, mask in severity_masks.items():
    area_img = mask.multiply(pixel_area_acres)
    area = area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=mtbs_2021_fires.geometry(), # confine the calculations to the MTBS 2021 fire perimeters
        scale=30,
        maxPixels=1e8
    ).getInfo()
    burn_area_stats[name] = list(area.values())[0]

total_burned_area = sum(burn_area_stats.values())


# Add pre-, post-fire, and burn severity layers to map
m.addLayer(prefire_rgb, {"bands": ["B4", "B3", "B2"], "min": 0.01, "max": 0.18}, "Pre-Fire RGB")
m.addLayer(postfire_rgb, {"bands": ["B4", "B3", "B2"], "min": 0.01, "max": 0.18}, "Post-Fire RGB")
m.addLayer(classified_dNBR_clipped, {
    "min": 0,
    "max": 6,
    "palette": burn_colors,
}, "Burn Severity Classification")

severity_leg = {
    "High Severity": "#a41fd6",
    "Moderate-High": "#ff641b",
    "Moderate": "#ffaf38",
    "Low": "#fff70b",
    "Unburned": "#0ae042",
    "Enhanced Regrowth": "#acbe4d",
    "High Regrowth": "#7a8737"
}
m.add_legend(title="Burn Severity", legend_dict=severity_leg, position="bottomleft")

# Add county and MTBS fire Boundaries
county_outline = affected_counties.style(**{
    "color": "black",
    "fillColor": "00000000",
    "width": 0.7
})
fire_outline = mtbs_2021_fires.style(**{
    "color": "red",
    "fillColor": "00000000",
    "width": 0.7
})
m.add_labels(affected_counties, "NAME", font_size="10pt", font_color="white", layer_name="County labels")
m.addLayer(county_outline, {}, "Affected Counties")
m.addLayer(fire_outline, {}, "MTBS 2021 fires extent")

# Add total burn area text box to map 
county_names = affected_counties.aggregate_array("NAME").getInfo()
county_list_str = ", ".join(county_names)

html_text = f"""
<div style="background-color:white; padding:10px; border-radius:4px; font-size:14px;">
  <strong>Estimated total burned area:</strong><br>
  {total_burned_area:.2f} hectares
</div>
"""
m.add_html(html_text, position="bottomright")
m.to_html("burn_severity_map.html")



# plot burn severity on bar chart
fig = go.Figure(data=[
    go.Bar(
        x=list(burn_area_stats.keys()),
        y=list(burn_area_stats.values()),
        marker_color=["#fff70b", "#ffaf38", "#ff641b", "#a41fd6"],
        hovertemplate='%{x}<br>Burned Area: %{y:.2f} hectares<extra></extra>'
    )
])
fig.update_layout(
    xaxis_title="Burn Severity",
    yaxis_title="Burned Area (ha)",
    height=400,
    width=600,
    margin=dict(l=40, r=40, t=60, b=40),
    font=dict(size=

fig.show()

# Save chart as an html file
fig.write_html("burn_severity_chart.html")

In [ ]:
# -----------------------------------------------
# Section B: Vegetation Recovery (2020–2024)
# -----------------------------------------------

season_dict = {
    "2020": ("2020-08-01", "2020-08-31"),
    "2021": ("2021-10-15", "2021-11-15"),
    "2022": ("2022-08-01", "2022-08-31"),
    "2023": ("2023-08-01", "2023-08-31"),
    "2024": ("2024-08-01", "2024-08-31"),
}

severity_classes = {
    "High": 0,
    "Moderate-High": 1,
    "Moderate-Low": 2,
    "Low": 3,
    "Unburned": 4
}

features = []

# Compute median NDVI and EVI per burn severity class
for year, (start, end) in season_dict.items():
    # Load and process S30 data
    s30 = process_s30(load_hls_data("NASA/HLS/HLSS30/v002", start, end)).select(["B2", "B4", "B5"])
    composite = s30.median()

    # Compute NDVI
    ndvi = composite.normalizedDifference(["B5", "B4"]).rename("NDVI")

    # Compute EVI
    evi = composite.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))",
        {
            "NIR": composite.select("B5"),
            "RED": composite.select("B4"),
            "BLUE": composite.select("B2")
        }
    ).rename("EVI")

    combined = ee.Image.cat([ndvi, evi])

    for label, value in severity_classes.items():
        mask = classified_dNBR.eq(value)
        masked = combined.updateMask(mask)

        stats = masked.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=mtbs_2021_fires.geometry(),
            scale=30,
            maxPixels=1e8
        )

        feature = ee.Feature(None, {
            "Year": year,
            "Class": label,
            "Median_NDVI": stats.get("NDVI"),
            "Median_EVI": stats.get("EVI")
        })
        features.append(feature)

# NDVI and EVI feature collection
ndvi_evi_fc = ee.FeatureCollection(features)

# Export as a csv file
VI_task = ee.batch.Export.table.toDrive(
    collection=ndvi_evi_fc,
    description="NDVI_EVI_By_Class_2020to2024",
    fileFormat="CSV"
)
VI_task.start()

# Load NDVI/EVI data
df = pd.read_csv("C:/Users/henry/Documents/DixieFire_Proj/NDVI_EVI_By_Class_2020to2024.csv")
df["Year"] = df["Year"].astype(str)

# Exclude 'Unburned' class
df = df[df["Class"] != "Unburned"]

# Pivot table
pivot_df = df.pivot(index="Year", columns="Class", values="Median_NDVI")

# Color scheme aligned with map visualizations
class_colors = {
    "Low": "#fff70b",
    "Moderate-Low": "#ffaf38",
    "Moderate-High": "#ff641b",
    "High": "#a41fd6"
}

# Create interactive NDVI plot by burn severity class
fig = go.Figure()

for cls, color in class_colors.items():
    if cls in pivot_df.columns:
        fig.add_trace(go.Scatter(
            x=pivot_df.index,
            y=pivot_df[cls],
            mode="lines+markers",
            name=cls,
            line=dict(color=color),
            marker=dict(size=8),
            hovertemplate=f"{cls}<br>Year: %{{x}}<br>NDVI: %{{y:.3f}}<extra></extra>"
        ))

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="NDVI",
    template="plotly_white",
    height=400,
    width=700,
    legend_title="Burn Severity"
)

fig.show()

# Save chart as an html file
fig.write_html("NDVI_regrowthMonitoring.html")



# Visualization parameters for Dynamic World classes
dw_vis = {
    'min': 0,
    'max': 8,
    'palette': [
        "#419BDF", "#397D49", "#88B053", "#7A87C6",
        "#E49635", "#DFC35A", "#C4281B", "#A59B8F", "#B39FE1"
    ]
}

# Load seasonal Dynamic World mode image composites for each year
images = {year: load_dw_mode(year) for year in range(2020, 2025)}

# Create image collection for time slider visualization
dw_seasonal_ic = ee.ImageCollection([images[year] for year in range(2020, 2025)])

# Add time slider and legend to map
m.add_time_slider(
    dw_seasonal_ic,
    vis_params=dw_vis,
    label="Dynamic World Seasonal Mode (2020–2024)"
)

# Define class labels and colors for legend
dw_labels = [
    "Water", "Trees", "Grass", "Flooded Vegetation",
    "Crops", "Shrub & Scrub", "Built", "Bare", "Snow & Ice"
]
dw_colors = dw_vis['palette']
dw_legend_dict = dict(zip(dw_labels, dw_colors))

# Add legend to map
m.add_legend(title="Dynamic World Classes", legend_dict=dw_legend_dict, position="bottomleft")


# Define target classes for land cover change analysis 
target_classes = {
    1: "Trees",
    2: "Grass",
    5: "Shrub_Scrub",
    7: "Bare"
}

# Compute area per class for a given year
def compute_area_for_year(year):
    mode_img = load_dw_mode(year)
    class_area_imgs = [
        mode_img.eq(class_id).multiply(ee.Image.pixelArea()).rename(class_name)
        for class_id, class_name in target_classes.items()
    ]
    stacked_img = ee.Image.cat(class_area_imgs)

    area_dict = stacked_img.reduceRegion(
        reducer=ee.Reducer.sum().forEachBand(stacked_img),
        geometry=mtbs_2021_fires,
        scale=30,
        maxPixels=1e13
    )

    return [
        ee.Feature(None, {
            "Year": year,
            "Class": class_name,
            "Area_ha": ee.Number(area_dict.get(class_name)).divide(10_000)
        })
        for class_name in target_classes.values()
    ]

# Compute transition matrix between two years
def compute_transition_matrix(img1, img2, year1, year2):
    transitions = img1.multiply(10).add(img2)
    results = []

    for from_class, to_class in [(i, j) for i in target_classes for j in target_classes]:
        code = from_class * 10 + to_class
        area = transitions.eq(code).multiply(ee.Image.pixelArea()).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=mtbs_2021_fires,
            scale=30,
            maxPixels=1e13
        ).get('label_mode')

        results.append({
            'From_Class': target_classes[from_class],
            'To_Class': target_classes[to_class],
            'Year_From': year1,
            'Year_To': year2,
            'Area_ha': ee.Number(area).divide(10_000)
        })

    return ee.FeatureCollection([ee.Feature(None, r) for r in results])

# Aggregate area and transition features
all_area_features = sum([compute_area_for_year(year) for year in range(2020, 2025)], [])
transition_fc_list = [
    compute_transition_matrix(images[y1], images[y2], y1, y2)
    for y1, y2 in zip(range(2020, 2024), range(2021, 2025))
]
all_transitions = ee.FeatureCollection(transition_fc_list).flatten()

# Export to csv
ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection(all_area_features),
    description="ClassArea_2020_2024", # area per year csv
    fileFormat="CSV"
).start()

ee.batch.Export.table.toDrive(
    collection=all_transitions,
    description="LandCoverTransitions_2020_2024", # land cover transition btn years csv
    fileFormat="CSV"
).start()

# Load and process transition data
df = pd.read_csv("LandCoverTransitions_2020_2024.csv")
df["source"] = df["From_Class"] + "_" + df["Year_From"].astype(str)
df["target"] = df["To_Class"] + "_" + df["Year_To"].astype(str)

# Map labels to indices
labels = pd.unique(df[["source", "target"]].values.ravel())
label_to_index = {label: i for i, label in enumerate(labels)}
df["source_idx"] = df["source"].map(label_to_index)
df["target_idx"] = df["target"].map(label_to_index)

# Define class colors
dw_class_colors = {
    "Trees": "#397D49",
    "Grass": "#88B053",
    "Shrub_Scrub": "#DFC35A",
    "Bare": "#A59B8F"
}

# Assign node colors based on class
def get_class_color(label):
    return next((color for class_name, color in dw_class_colors.items() if label.startswith(class_name)), "#CCCCCC")

node_colors = [get_class_color(label) for label in labels]

# Shorten labels for Sankey display
def shorten_label(label):
    class_part, year = label.rsplit("_", 1)
    return f"{'Shrub' if class_part == 'Shrub_Scrub' else class_part} {year}"

short_labels = [shorten_label(label) for label in labels]

# Compute percentage change per source node
source_totals = df.groupby("source")["Area_ha"].sum().to_dict()
df["Percentage"] = df.apply(lambda row: round((row["Area_ha"] / source_totals[row["source"]]) * 100, 2), axis=1)

# Create hover tooltips
df["hover_text"] = df.apply(
    lambda row: f"{row['From_Class']} → {row['To_Class']} ({row['Year_From']}→{row['Year_To']})<br>{row['Area_ha']:.2f} ha ({row['Percentage']}%)",
    axis=1
)

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=short_labels,
        color=node_colors
    ),
    link=dict(
        source=df["source_idx"],
        target=df["target_idx"],
        value=df["Area_ha"],
        customdata=df["hover_text"],
        hovertemplate="%{customdata}<extra></extra>"
    )
)])

# Title
fig.update_layout(title_text="Land Cover Change from the Dixie Fire (2020–2024)", font_size=12)

fig.show()

# Export as an html file
fig.write_html("land_cover_sankey.html")